# Find the Most Frequent Words with Mismatches in a String

[ba1i](https://rosalind.info/problems/ba1i/)

We defined a mismatch in “Compute the Hamming Distance Between Two Strings”. We now generalize “Find the Most Frequent Words in a String” to incorporate mismatches as well.

Given strings Text and Pattern as well as an integer d, we define Countd(Text, Pattern) as the total number of occurrences of Pattern in Text with at most d mismatches. For example, Count1(AACAAGCTGATAAACATTTAAAGAG, AAAAA) = 4 because AAAAA appears four times in this string with at most one mismatch: AACAA, ATAAA, AAACA, and AAAGA. Note that two of these occurrences overlap.

A most frequent k-mer with up to d mismatches in Text is simply a string Pattern maximizing Countd(Text, Pattern) among all k-mers. Note that Pattern does not need to actually appear as a substring of Text; for example, AAAAA is the most frequent 5-mer with 1 mismatch in AACAAGCTGATAAACATTTAAAGAG, even though AAAAA does not appear exactly in this string. Keep this in mind while solving the following problem.

### Frequent Words with Mismatches Problem

Find the most frequent k-mers with mismatches in a string.

    Given: 

A string Text as well as integers k and d.

    Return: 

All most frequent k-mers with up to d mismatches in Text.

Sample Dataset

    ACGTTGCATGTCGCATGATGCATGAGAGCT
    4 1

Sample Output

    GATG ATGC ATGT

In [80]:
from itertools import product

In [81]:
def get_hamming_distance(dna1, dna2):
    return sum([x != y for x, y in zip(dna1, dna2)])

In [82]:
def get_frequent_words(dna, k, d):
    kmers = {''.join(kmer):0 for kmer in product('ACGT', repeat = k)} 
    max_count = 0
    for i in range(len(dna) - k + 1):
        for kmer in kmers.keys():
            if get_hamming_distance(dna[i:i+k], kmer) <= d:
                kmers[kmer] += 1
                if max_count < kmers[kmer]:
                    max_count = kmers[kmer]
    
    return ' '.join([key for (key, value) in kmers.items() if value == max_count])

In [83]:
def symbol_to_number(symbol):
    symbols = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    return symbols[symbol]

In [84]:
def number_to_symbol(number):
    return 'ACGT'[number]

In [85]:
def pattern_to_number(pattern):
    if len(pattern) == 0:
        return 0
    return 4 * pattern_to_number(pattern[:-1]) + symbol_to_number(pattern[-1])

In [86]:
def get_quotient(index):
    return index // 4

In [87]:
def get_remainder(index):
    return index % 4

In [88]:
def number_to_pattern(index, k):
    if k == 1:
        return number_to_symbol(index)
    prefix_index = get_quotient(index) 
    symbol = number_to_symbol(get_remainder(index))
    prefix_pattern = number_to_pattern(prefix_index, k-1)
    return prefix_pattern + symbol

In [89]:
def get_neighbors(dna, d):
    if d == 0:
        return {dna}
    if len(dna) == 1:
        return {'A','C','G','T'}
    neighborhood = set()
    suffix_neighbors = get_neighbors(dna[1:], d)
    for pattern in suffix_neighbors:
        if get_hamming_distance(dna[1:], pattern) < d:
            for nucleotide in pattern:
                neighborhood.add(nucleotide+pattern)
        else:
            neighborhood.add(dna[1]+pattern)
    return neighborhood

In [90]:
def get_approximate_pattern_count(seq, pattern, d):
    count = 0
    for i in range(len(seq) - len(pattern) + 1):
        if get_hamming_distance(pattern, seq[i: i + len(pattern)]) <= d:
            count += 1 
    return count

In [93]:
def get_faster_frequent_words(dna, k, d):
    freq_patterns = set()
    freq_array = [0] * 4**k
    close = [0] * 4**k
    for i in range(len(dna) - k + 1):
        neighborhood = get_neighbors(dna[i:i+k], d)
        for pattern in neighborhood:
            index = pattern_to_number(pattern)
            close[index] = 1
    for i in range(4**k):
        if close[i] == 1:
            pattern = number_to_pattern(i, k)
            freq_array[i] = get_approximate_pattern_count(dna, pattern, d)
    max_count = max(freq_array)
    for i in range(4**k):
        if freq_array[i] == max_count:
            pattern = number_to_pattern(i, k)
            freq_patterns.add(pattern)
    return ' '.join(freq_patterns)

In [94]:
# file = "rosalind_ba1i.txt" 
file = "input.txt" 
with open(file, 'r') as f:
    lines = f.readlines()
    dna  = lines[0].strip()
    k, d = map(int, lines[1].split())

print(get_frequent_words(dna, k, d))
print(get_faster_frequent_words(dna, k, d))

ATGC ATGT GATG
GATG ATGT ATGC
